In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
from datetime import date, timedelta
import urllib
import json

In [2]:
#טעינת הקבצים באמצעות API
url = 'https://data.gov.il/api/action/datastore_search?resource_id=823479b4-4771-43d8-9189-6a2a1dcaaf10&limit=10000'
file = urllib.request.urlopen(url)
data = json.loads(file.read())
DeadSea_Levels_df = pd.DataFrame(data['result']['records'])
DeadSea_Levels_df = DeadSea_Levels_df.rename(columns={'תאריך מדידה':'Test_Date','מפלס':'DeadSea_Level'}).drop(columns='_id')
DeadSea_Levels_df['DeadSea_Level'] = DeadSea_Levels_df['DeadSea_Level'].astype('float')
#מיינו לפי תאריך
DeadSea_Levels_df['Test_Date'] = pd.to_datetime(DeadSea_Levels_df['Test_Date'])


DeadSea_Levels_df = DeadSea_Levels_df.sort_values(by='Test_Date').reset_index(drop=True)

In [3]:
DeadSea_Levels_df.to_csv('DeadSea_levels_Scrapping.csv', index=False)

In [4]:
DeadSea_Levels_df.head() #בדיקת תקינות הטבלה בחמשת השורות הראשונות

,Test_Date,DeadSea_Level
0,1976-01-10,-398.95
1,1976-01-11,-399.01
2,1976-01-12,-399.12
3,1977-01-01,-399.05
4,1977-01-02,-399.04


In [5]:
DeadSea_Levels_df.tail() #בדיקת תקינות חמשת השורות האחרונות

,Test_Date,DeadSea_Level
627,2021-03-10,-436.16
628,2021-04-29,-435.44
629,2021-05-31,-435.55
630,2021-08-29,-436.00
631,2021-12-29,-436.39


In [6]:
DeadSea_Levels_df.info() #מידע על משתני הטבלה

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 632 entries, 0 to 631
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Test_Date      632 non-null    datetime64[ns]
 1   DeadSea_Level  632 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 10.0 KB


In [7]:

def call_api(page_number):
    """
    הפונקציה מקבלת מספר של עמוד ומייצרת קריאה,
    בודקת את סטטוס הקריאה בכל פעם, במידה והסטטוס הוא 200 ממשיכים בתקינות הפונקציה,
    אחרת אנו שולחים קריאה חדשה לאחר 60 שניות
    """
    url = f'https://data.gov.il/api/3/action/datastore_search?resource_id=cee3ad4a-4e77-4015-8245-52505417d7ea&offset={page_number}'  
    res = requests.get(url)
    if res.status_code == 200:
        return json.loads(res.content)
    else:
        print(f'[{res.status_code}]Something is wrong on {page_number}, Reset in 60 seconds...')
        time.sleep(60)
        return call_api(page_number)
    
def load_datajson(path):
    data = pd.read_json(path)
    data.to_csv('data_as_csv.csv')
    

def save_data(data, json_name):
    # טוען את הדטהבייס הקודם  
    with open(json_name,'r') as data_json:
         json_data = json.load(data_json)
    
    # מוסיפים את הדאטה החדש, לקובץ הקיים.
    with open(json_name,'w') as new_update:
        json.dump(data,new_update,indent=4)
    return len(json_data)

# אנו טוענים את הדאטה ושומרים אותו בקובץ database.json
database = [] 
max_number = 1474457 # הכמות המקסימלית של הנתונים
start_number = 0
done = 0    # כמות העמודים שביצענו עליהם את המשיכה כבר.
for page_num in range(start_number,max_number,100):
    done =+ page_num  #הוספת המספר של העמוד הנוכחי למשתנה העזר שלנו 
    data_requests = call_api(page_num)['result']['records']
    for record in data_requests:
        database.append(record)
    if (done % 10000) == 0: # אחרי כל 10,000 אנו שומרים את הדאטה
        print(f'Done till now: {done}/{max_number}') # הדפסה לכמה הגענו עד עתה
        save_data(database,'database.json') # שמירת הנתונים בתוך הקובץ.
        
print('FINALLY DONE!')
load_datajson('database.json')
print('DATA SAVED!!')

Done till now: 0/1474457
Done till now: 10000/1474457
Done till now: 20000/1474457
Done till now: 30000/1474457
Done till now: 40000/1474457
Done till now: 50000/1474457
Done till now: 60000/1474457
Done till now: 70000/1474457
Done till now: 80000/1474457
Done till now: 90000/1474457
Done till now: 100000/1474457
Done till now: 110000/1474457
Done till now: 120000/1474457
Done till now: 130000/1474457
Done till now: 140000/1474457
Done till now: 150000/1474457
Done till now: 160000/1474457
Done till now: 170000/1474457
Done till now: 180000/1474457
Done till now: 190000/1474457
Done till now: 200000/1474457
Done till now: 210000/1474457
Done till now: 220000/1474457
Done till now: 230000/1474457
Done till now: 240000/1474457
Done till now: 250000/1474457
Done till now: 260000/1474457
Done till now: 270000/1474457
Done till now: 280000/1474457
Done till now: 290000/1474457
Done till now: 300000/1474457
Done till now: 310000/1474457
Done till now: 320000/1474457
Done till now: 330000/14

ConnectionError: HTTPSConnectionPool(host='data.gov.il', port=443): Max retries exceeded with url: /api/3/action/datastore_search?resource_id=cee3ad4a-4e77-4015-8245-52505417d7ea&offset=1198000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001826B77A5B0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [8]:
max_number = 1474457 # הכמות המקסימלית של הנתונים
start_number = 1190000
done = 0    # כמות העמודים שביצענו עליהם את המשיכה כבר.
for page_num in range(start_number,max_number,100):
    done =+ page_num  #הוספת המספר של העמוד הנוכחי למשתנה העזר שלנו 
    data_requests = call_api(page_num)['result']['records']
    for record in data_requests:
        database.append(record)
    if (done % 10000) == 0: # אחרי כל 10,000 אנו שומרים את הדאטה
        print(f'Done till now: {done}/{max_number}') # הדפסה לכמה הגענו עד עתה
        save_data(database,'database.json') # שמירת הנתונים בתוך הקובץ.
        
print('FINALLY DONE!')
load_datajson('database.json')
print('DATA SAVED!!')

Done till now: 1190000/1474457
Done till now: 1200000/1474457
Done till now: 1210000/1474457
Done till now: 1220000/1474457
Done till now: 1230000/1474457
Done till now: 1240000/1474457
Done till now: 1250000/1474457
Done till now: 1260000/1474457
Done till now: 1270000/1474457
Done till now: 1280000/1474457
Done till now: 1290000/1474457
Done till now: 1300000/1474457
Done till now: 1310000/1474457
Done till now: 1320000/1474457
Done till now: 1330000/1474457
Done till now: 1340000/1474457
Done till now: 1350000/1474457
Done till now: 1360000/1474457
Done till now: 1370000/1474457
Done till now: 1380000/1474457
Done till now: 1390000/1474457
Done till now: 1400000/1474457
Done till now: 1410000/1474457
Done till now: 1420000/1474457
Done till now: 1430000/1474457
Done till now: 1440000/1474457
Done till now: 1450000/1474457
Done till now: 1460000/1474457
Done till now: 1470000/1474457
FINALLY DONE!
DATA SAVED!!


In [10]:
daily_Data_df = pd.read_csv("data_as_csv.csv")

In [15]:
daily_Data_df.head()

,Unnamed: 0,_id,stn_num,time_obs,tmp_air_max,tmp_air_min,tmp_grass_min,evp_cls_a_12,evp_cls_a_18,evp_cls_a_06_next,...,rpr_hail,rpr_snow,rpr_frost,rpr_dew,rpr_fog,rpr_mist,rpr_thunder,rpr_lightening,rpr_sand_storm,rpr_gale
0,0,1,9029,1978-04-01T00:00:00,232.0,97.0,70.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,9029,1978-04-02T00:00:00,260.0,92.0,60.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,9029,1978-04-03T00:00:00,242.0,138.0,105.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,9029,1978-04-04T00:00:00,306.0,100.0,65.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,9029,1978-04-05T00:00:00,245.0,160.0,148.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
